# CoppeliaSim: Python interface
### Setup
Make sure you have following files in your directory, in order to run the various examples:

1. sim.py and simConst.py in coppelia's folder ```programming/remoteApiBindings/python/python
2. the appropriate remote API library: "remoteApi.dll" (Windows), "remoteApi.dylib" (Mac) or "remoteApi.so" (Linux). In coppelia's folder ```programming/remoteApiBindings/lib/lib/


### Before executing



* Make sure to have the server side running in CoppeliaSim.

* In a child script of a CoppeliaSim scene, add following command to be executed just once, at simulation start:
```python
simRemoteApi.start(19999)
```

* then start simulation, and run this program.

* For each successful call to simxStart, there should be a corresponding call to simxFinish at the end!


### Tutorials
* Python tutorial: http://34.208.13.223/VREP/04PythonTutorial/
* CoppeliaSim tutorial: https://www.youtube.com/playlist?list=PLjzuoBhdtaXOoqkJUqhYQletLLnJP8vjZ
* API Documentation: https://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

# Scene
For testing purposes, you can use the file of the class: three_robots.ttt.
However, I recommend to start with an empty scene and add the robots.


In [5]:
# Import the python library that connects to CoppeliaSim, the file is sim.py.
try:
    import sim
except:
    print('--------------------------------------------------------------')
    print('"sim.py" could not be imported. This means very probably that')
    print('either "sim.py" or the remoteApi library could not be found.')
    print('Make sure both are in the same folder as this file,')
    print('or appropriately adjust the file "sim.py"')
    print('--------------------------------------------------------------')
    print('')

import time

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check if python is connecting to Coppelia

In [7]:
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

Program started
Connected to remote API server
Program ended


# Controlling motors

1. Add a Pioneer_p3dx robot.
2. Remove its Lua script.
3. Start the remote API server
4. Send a control command with the following script

In [190]:
# Actuator names
left_motor_name = "Pioneer_p3dx_leftMotor"
right_motor_name = "Pioneer_p3dx_rightMotor"

# Values to send  (rad/sec)
left_velocity = -1.0
right_velocity = 1.0


# Send the command!
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)
    
    # Get handlers or actuators
    err_code, l_motor_handle = sim.simxGetObjectHandle(clientID, left_motor_name, sim.simx_opmode_blocking)
    err_code, r_motor_handle = sim.simxGetObjectHandle(clientID, right_motor_name, sim.simx_opmode_blocking)

    # Send the values!
    err_code = sim.simxSetJointTargetVelocity(clientID, l_motor_handle, left_velocity, sim.simx_opmode_streaming)
    err_code = sim.simxSetJointTargetVelocity(clientID, r_motor_handle, right_velocity, sim.simx_opmode_streaming)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

Program started
Connected to remote API server
Program ended


# Object oriented programming

In [18]:
class robot():
    
    def __init__(self, motor_names, frame_name=None):        
        self.client_id = self.open_connection()
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  None
        if frame_name is not None:
            self.frame = self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object):
        relative_handle = self._get_handler(relative_object)
        sim.simxSetObjectPosition(clientID, self.frame, relative_handle, position, sim.simx_opmode_oneshot)

In [14]:
# Joint names in coppelia for KUKA omnidirectional robot
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']


r = robot(motor_names)  # Create an instance of our robot
r.send_motor_velocities([1., -1., -1., 1.])  # Move the robot

r.close_connection()  # End

Robot connected
Connection closed


# Control loop

In [16]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot(motor_names)  # Create an instance of our robot

time_steps = linspace(0, 6*pi, 100)



for t in time_steps:
    if r.client_id == -1:  # Check connection
        print('Robot is not connected')
        break
    
    vx, vy = -sin(t), cos(t)
    r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    time.sleep(0.1)

    
r.send_motor_velocities([0, 0, 0, 0])

r.close_connection()  # End

Robot connected
Connection closed


# Drone control

In [27]:
r = robot([], frame_name='Quadricopter_target')
r.set_position([0,0,1], 'Cuboid')
r.close_connection()

Robot connected
Connection closed


### Lets make the drone to follow a circle
* Add a primitive shape -> cuboid with the name 'Cuboid'. Then the drone can move around it in circles

In [28]:
r = robot([], frame_name='Quadricopter_target')

time_steps = linspace(0, 2*pi, 100)



for t in time_steps:
    if r.client_id == -1:  # Check connection
        print('Robot is not connected')
        break
    
    px, py = cos(t), sin(t)
    r.set_position([px,py,1], 'Cuboid')
    
    
    time.sleep(0.01)

    


r.close_connection()  # End

Robot connected
Connection closed
